# Assignment 8 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [26]:
#import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler;
from sklearn.linear_model import LogisticRegression;

## Part [a]

In [27]:
imgs = np.load("mnist.npz")

#Flatten images
X_train = imgs['x_train'].reshape((60000, 28 * 28))
X_test = imgs['x_test'].reshape((10000, 28 * 28))

#scale their features
scale = StandardScaler().fit(X_train)
X_train_scaled = scale.transform(X_train)
X_test_scaled = scale.transform(X_test)
y_train = imgs['y_train']
y_test = imgs['y_test']

## Part [b]

In [32]:
#Create logistic regressor
log_reg = LogisticRegression().fit(X_train_scaled, y_train);

#Calculate classification accuracy on train and test sets
train_accuracy = log_reg.score(X_train_scaled, y_train);
test_accuracy = log_reg.score(X_test_scaled, y_test);

c:\Users\Dennis Goldenberg\anaconda\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
#Report train and test accuracy
print("Train Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)

Train Accuracy:  0.94345
Test Accuracy:  0.9246


## Part [c]

## Part [d]